In [1]:
import logging
import os
import sys
from datetime import datetime

import pandas as pd
import yfinance as yf
from pandas_datareader import data as pdr
from pyrate_limiter import Duration, Limiter, RequestRate
from requests import Session
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket

yf.pdr_override()

In [2]:
notebooks_dir_parent = os.path.relpath(os.path.dirname(os.getcwd()))
sm_dir = os.path.join(notebooks_dir_parent, "stock-market-analysis", "src")

sys.path.append(sm_dir)

In [3]:
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    def __init__(self, timeout=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.timeout = timeout

    def request(self, *args, **kwargs):
        # Pass the stored timeout to the request method
        kwargs["timeout"] = self.timeout
        return super().request(*args, **kwargs)


session = CachedLimiterSession(
    limiter=Limiter(RequestRate(2, Duration.SECOND * 5)),  # max 2 requests per 5 seconds
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache("yfinance.cache"),
    timeout=5,
)


def get_logger(name):
    # Create a logger
    logger = logging.getLogger(name)

    # Set the logging level (adjust as needed)
    logger.setLevel(logging.DEBUG)

    # Create a console handler and set the level
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)

    # Create a formatter and add it to the handler
    formatter = logging.Formatter("%(asctime)s | %(levelname)s:%(name)s - %(message)s")
    ch.setFormatter(formatter)

    # Add the handler to the logger
    logger.addHandler(ch)
    return logger


logger = get_logger("notebook")

In [4]:
data = pdr.get_data_yahoo(
    "ADANIENT.NS", end=datetime.today(), session=session, interval="1m"
).reset_index()
data

[*********************100%%**********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2024-01-24 09:15:00+05:30,2901.050049,2928.800049,2901.050049,2920.000000,2920.000000,0
1,2024-01-24 09:16:00+05:30,2920.100098,2926.899902,2918.649902,2924.449951,2924.449951,10445
2,2024-01-24 09:17:00+05:30,2924.250000,2924.250000,2911.000000,2915.050049,2915.050049,15317
3,2024-01-24 09:18:00+05:30,2913.050049,2913.050049,2902.800049,2905.100098,2905.100098,15000
4,2024-01-24 09:19:00+05:30,2902.199951,2903.899902,2885.149902,2887.449951,2887.449951,15909
...,...,...,...,...,...,...,...
1485,2024-01-30 15:25:00+05:30,3095.250000,3096.000000,3093.600098,3096.000000,3096.000000,8138
1486,2024-01-30 15:26:00+05:30,3095.899902,3097.750000,3094.750000,3097.649902,3097.649902,10538
1487,2024-01-30 15:27:00+05:30,3098.000000,3098.000000,3095.000000,3096.699951,3096.699951,19169
1488,2024-01-30 15:28:00+05:30,3097.250000,3097.449951,3095.500000,3097.000000,3097.000000,15108


In [19]:
from models import TickerSummaryModel

def get_ticker_info(ticker_code: str):
    # Create a Ticker object using yfinance with the provided ticker code and session
    ticker_info = yf.Ticker(ticker_code, session=session)

    # Retrieve information about the ticker
    ticker_info = ticker_info.info
    if "symbol" in ticker_info:
        ticker_info = TickerSummaryModel(
            ticker_code = ticker_info["symbol"],
            name = ticker_info["shortName"],
            exchange = ticker_info["exchange"]
        )
        return ticker_info.model_dump()
    else:
        logger.error("Invalid Ticker code '%s'", ticker_code)
        # raise ValueError("Invalid Ticker code '%s'", ticker_code)


get_ticker_info("AAPL")

{'name': 'Apple Inc.', 'ticker_code': 'AAPL', 'exchange': 'NMS'}

In [6]:
datetime.strptime("2015-01-01", "%Y-%m-%d")

datetime.datetime(2015, 1, 1, 0, 0)

In [33]:
ticker_list = pd.read_csv("us_symbols.csv", keep_default_na=False)
ticker_list.rename(columns={"ticker": "ticker_code"}, inplace=True)
ticker_list

,ticker_code,name,exchange
0,A,Agilent Technologies Inc.,NYSE
1,AA,Alcoa Corporation,NYSE
2,AACG,ATA Creativity Global,NASDAQ
3,AACI,Armada Acquisition Corp. I,NASDAQ
4,AACT,Ares Acquisition Corporation II,NYSE
...,...,...,...
5674,ZVRA,Zevra Therapeutics Inc.,NASDAQ
5675,ZVSA,ZyVersa Therapeutics Inc.,NASDAQ
5676,ZWS,Zurn Elkay Water Solutions Corporation,NYSE
5677,ZYME,Zymeworks Inc.,NASDAQ


In [34]:
from models import TickerBaseListModel, TickerSummaryModel
from db import tickers_info_collection

ticker_list["created_at"] = datetime.utcnow()
ticker_list = TickerBaseListModel(ticker_list.to_dict(orient="records")).model_dump()
# tickers_info_collection.insert_many(ticker_list)
# ticker_list

In [35]:
from db import PyObjectId
from models import TickerSummaryModel
TickerSummaryModel(**tickers_info_collection.find_one({"_id": PyObjectId("65b9f531e4178f1bf53b7f4c")},{"_id": 0}))

TickerSummaryModel(name='Agilent Technologies Inc.', ticker_code='A', exchange='NYSE')

In [9]:
from manager import PortfolioManager
from models import PortfolioModel

pm = PortfolioManager(username="john_doe")

# pm.create_portfolio(
#     PortfolioModel(
#         username=pm.username,
#         portfolio_name="portfolio1",
#         # tickers: Optional[List[TickerSummaryModel]] = None
#         created_at=datetime.utcnow()
#     )
# )

# pm.create_portfolio(
#     PortfolioModel(
#         username=pm.username,
#         portfolio_name="portfolio2",
#         # tickers: Optional[List[TickerSummaryModel]] = None
#         created_at=datetime.utcnow()
#     )
# )

pm.get_portfolios().model_dump()

[{'portfolio_name': 'portfolio1',
  'tickers': None,
  'created_at': datetime.datetime(2024, 1, 31, 8, 53, 51, 514000),
  'updated_at': None},
 {'portfolio_name': 'portfolio2',
  'tickers': None,
  'created_at': datetime.datetime(2024, 1, 31, 8, 54, 28, 279000),
  'updated_at': None}]

In [21]:
x = PortfolioModel(
        username=pm.username,
        portfolio_name="portfolio2",
        # tickers: Optional[List[TickerSummaryModel]] = None
        created_at=datetime.utcnow()
    )
x

PortfolioModel(username='john_doe', portfolio_name='portfolio2', tickers=None, created_at=datetime.datetime(2024, 1, 31, 11, 12, 41, 849866), updated_at=None)

In [24]:
from models import PortfolioPreviewModel
PortfolioPreviewModel(**x.model_dump())

PortfolioPreviewModel(portfolio_name='portfolio2', tickers=None, created_at=datetime.datetime(2024, 1, 31, 11, 12, 41, 849866), updated_at=None)